In [1]:
# common packages
import pandas as pd
import time

# # DL framework
import torch
from torchtext import data

from attractivedata import AttractiveData
from trainer import AttractiveTrainer

## Load and prepare data

In [2]:
train_file = 'data/train.csv'
test_file = 'data/test.csv'
pretrained_file = 'glove.42B.300d'
max_size = 64
min_freq = 5
batch_size = 64

In [3]:
AttractiveData = AttractiveData(train_file, test_file, pretrained_file, max_size, min_freq, batch_size)

In [4]:
for i, sentence in enumerate(AttractiveData.test_data):
    if i == 3:
        print(vars(AttractiveData.train_data[i]), vars(sentence))

{'Headline': ['Sorry', ',', 'i', 'spent', 'it', 'on', 'myself', '!', 'Harvey', 'Nichols', "'", 'hilarious', 'Christmas', 'advert', 'sees', 'people', 'treating', 'themselves', 'instead', 'of', 'others'], 'Category': 'femail', 'Label': '3.333333333333333'} {'Headline': ['Three', 'police', 'officers', 'accused', 'of', 'stealing', '?', '?', '30k', 'during', 'raid', 'on', 'criminal'], 'Category': 'news'}


In [5]:
len(AttractiveData.CATEGORIES_LABEL.vocab.freqs)

22

In [6]:
AttractiveData.LABEL.vocab.freqs

Counter({'4.0': 226,
         '2.333333333333333': 194,
         '4.5': 43,
         '3.333333333333333': 313,
         '3.6666666666666665': 260,
         '2.6666666666666665': 281,
         '2.0': 135,
         '2.5': 36,
         '1.6666666666666667': 28,
         '3.0': 354,
         '4.333333333333333': 82,
         '4.666666666666667': 29,
         '1.5': 16,
         '3.5': 22,
         '1.3333333333333333': 4,
         '1.0': 5,
         '5.0': 12})

## Start to train

In [7]:
timestr = time.strftime("%Y%m%d-%H%M%S")
save_name = './model/AttractiveNet'
num_workers = 10
input_dim = len(AttractiveData.TEXT.vocab)
embedding_dim = 300

category_dim = len(AttractiveData.CATEGORIES_LABEL.vocab)
category_output_dim = 16
hidden_dim = 256
output_dim = len(AttractiveData.LABEL.vocab.freqs)
log_steps = 10
epochs = 100
lr = 1e-3
num_layers = 3
nhead = 4
dropout = 0.1
pretrained_embeddings = AttractiveData.TEXT.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([1410, 300])


In [8]:
# max(AttractiveData.df_train.Headline.str.len()), max(AttractiveData.df_test.Headline.str.len())

In [9]:
AttractiveTrainer = AttractiveTrainer(save_name, log_steps, epochs, lr, timestr, AttractiveData.device, AttractiveData.trainloader, input_dim, category_dim, category_output_dim, embedding_dim, hidden_dim, output_dim, pretrained_embeddings, dropout, num_layers, nhead)

In [10]:
AttractiveTrainer.model

TransformerModel(
  (embedding): AttractiveEmbedding(
    (token): TokenEmbedding(1410, 300, padding_idx=0)
    (position): PositionalEmbedding()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (category_embedding): CategoryEmbedding(22, 16, padding_idx=0)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=300, out_features=300, bias=True)
        )
        (linear1): Linear(in_features=300, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=300, bias=True)
        (norm1): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn)

In [11]:
AttractiveTrainer.train()

Epoch 0
EP: train | lr: 0.001: 100%|| 32/32 [00:00<00:00, 172.28it/s]

EP_train | avg_loss: 2.8085429295897484 |
Epoch 1
EP: train | lr: 0.001: 100%|| 32/32 [00:00<00:00, 171.96it/s]

EP_train | avg_loss: 2.6118322014808655 |
Epoch 2
EP: train | lr: 0.001: 100%|| 32/32 [00:00<00:00, 172.03it/s]

EP_train | avg_loss: 2.5103530883789062 |
Epoch 3
EP: train | lr: 0.001: 100%|| 32/32 [00:00<00:00, 171.47it/s]

EP_train | avg_loss: 2.461367554962635 |
Epoch 4
EP: train | lr: 0.001: 100%|| 32/32 [00:00<00:00, 160.42it/s]

EP_train | avg_loss: 2.4221411123871803 |
Epoch 5
EP: train | lr: 0.001: 100%|| 32/32 [00:00<00:00, 147.44it/s]

EP_train | avg_loss: 2.4100786224007607 |
Epoch 6
EP: train | lr: 0.001: 100%|| 32/32 [00:00<00:00, 158.33it/s]

EP_train | avg_loss: 2.386109508574009 |
Epoch 7
EP: train | lr: 0.001: 100%|| 32/32 [00:00<00:00, 164.09it/s]

EP_train | avg_loss: 2.3840832710266113 |
Epoch 8
EP: train | lr: 0.001: 100%|| 32/32 [00:00<00:00, 171.97it/s]

EP_train | avg_loss: 2.3699

## for classification, not better

In [14]:
a = AttractiveTrainer.train_predict
AttractiveData.LABEL.vocab.itos[int(a[0])], AttractiveTrainer.train_true[0]
correct = 0
pred_list = []
true_list = []
for i in range(len(a)):
    pred = AttractiveData.LABEL.vocab.itos[int(a[i])]
    pred_list.append(float(pred))
    true = AttractiveData.LABEL.vocab.itos[int(AttractiveTrainer.train_true[i])]
    true_list.append(float(true))
mean_squared_error(true_list, pred_list)
# true_list

0.5601443355119825

## Below is testing

In [15]:
from transformermodel import TransformerModel
PATH = './model/AttractiveNet_20201029-134515_2.285.100'
load_model = TransformerModel(nhead, input_dim, category_dim, category_output_dim, embedding_dim, hidden_dim, output_dim, dropout, num_layers).to(AttractiveData.device)
load_model.load_state_dict(torch.load(PATH))
load_model.eval()

TransformerModel(
  (embedding): AttractiveEmbedding(
    (token): TokenEmbedding(1410, 300, padding_idx=0)
    (position): PositionalEmbedding()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (category_embedding): CategoryEmbedding(22, 16, padding_idx=0)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=300, out_features=300, bias=True)
        )
        (linear1): Linear(in_features=300, out_features=256, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=256, out_features=300, bias=True)
        (norm1): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn)

In [16]:
def predict_attractive(sentence, category):
    indexed_sentence = [AttractiveData.TEXT.vocab.stoi[t] for t in sentence]
    indexed_category = [AttractiveData.CATEGORIES_LABEL.vocab.stoi[category]]
    tensor_sentence = torch.LongTensor(indexed_sentence).to(AttractiveData.device)
    tensor_category = torch.LongTensor(indexed_category).to(AttractiveData.device)

    tensor_sentence = tensor_sentence.unsqueeze(1)
    tensor_category = tensor_category

    prediction = load_model(tensor_sentence, tensor_category)

    _, predict_class = torch.max(prediction, dim=1)
    # print(prediction)
    # print(predict_class)
    prediction = AttractiveData.LABEL.vocab.itos[int(predict_class.item())]
    
    return float(prediction)

In [17]:
predict_list = []
for i, sentence in enumerate(AttractiveData.test_data):
    prediction = predict_attractive(sentence.Headline, sentence.Category)
    predict_list.append(prediction)
AttractiveData.df_test['Label'] = predict_list
AttractiveData.df_test[['ID', 'Label']].to_csv('transformers.csv', index=False)

In [18]:
train_category = list(AttractiveData.CATEGORIES_LABEL.vocab.freqs)
test_category = list(AttractiveData.df_test['Category'].value_counts().keys())
for each_test in test_category:
    if each_test not in train_category:
        print(each_test)
print()
for each_train in train_category:
    if each_train not in test_category:
        print(each_train)


gardening
travelnews
golf
concussion
beauty
racing


In [17]:
train_category, test_category

(['travel',
  'health',
  'femail',
  'sport',
  'gardening',
  'sciencetech',
  'news',
  'food',
  'football',
  'travelnews',
  'cricket',
  'golf',
  'books',
  'rugbyunion',
  'home',
  'boxing',
  'tennis',
  'concussion',
  'othersports',
  'beauty',
  'formulaone',
  'racing'],
 ['health',
  'femail',
  'sciencetech',
  'travel',
  'news',
  'football',
  'food',
  'living',
  'books',
  'boxing',
  'rugbyunion',
  'othersports',
  'formulaone',
  'cricket',
  'us',
  'tennis',
  'sport',
  'middleeast',
  'racing'])

## Below just for fun guess

In [17]:
a = AttractiveData.df_test['ID'].to_list()

In [13]:
from sklearn.metrics import mean_squared_error

In [9]:
mean_squared_error(a, b)
# Training all 3.0 got mse = 0.5545

0.5545343137254902

In [18]:
# only for fun
import random
guess_list = [2.3333333333333335, 3.3333333333333335, 3.6666666666666665, 2.6666666666666665]
b = []
for i in range(len(a)):
    b.append(random.choice(guess_list))

In [19]:
AttractiveData.df_test['Label'] = b

In [20]:
AttractiveData.df_test[['ID', 'Label']].to_csv('all_3.csv', index=False)